<a href="https://colab.research.google.com/github/Raymond-223/ICPC-Algorithm-Knowledge/blob/main/Fundamentals_of_Dynamic_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#动态规划基础

##动态规划原理

能用动态规划解决的问题，需要满足三个条件：最优子结构，无后效性和子问题重叠。

###最优子结构
问题的最优解包含子问题的最优解。反过来说就是，我们可以通过子问题的最优解，来推导出原问题的最优解。

####证明逻辑

    1. 证明问题最优解的第一个组成部分是做出一个选择；

    2. 对于一个给定问题，在其可能的第一步选择中，假定你已经知道哪种选择才会得到最优解，而你现在无需纠结其获取方式；

    3. 给定可获得的最优解的选择后，确定这次选择会衍生哪些子问题以及如何最好地刻画子问题空间；

    4. 证明作为构成原问题最优解的组成部分，其每个子问题的解就是他本身的最优解。方法是反证法：考虑加入的某个子问题的解不是其自身的最优解，那么就可以从原问题的解中用该子问题的最优解替换掉当前的非最优解，从而得到原问题的一个更优的解，从而与原问题最优解的假设矛盾。

注意：具有最优子结构也可能是适合用贪心的方法求解，要确保我们考察了最优解中用到的所有子问题，且要保持子问题空间尽量简单，只在必要时扩展

###无后效性
已经求解的子问题，不会再受到后续决策的影响。

###子问题重叠
如果有大量的重叠子问题，我们可以用空间将这些子问题的解存储下来，避免重复求解相同的子问题，从而提升效率。

###动态规划基本思路
对于一个能用动态规划解决的问题，一般采用如下思路解决：

    1. 将原问题划分为若干阶段，每个阶段对应若干个子问题，提取这些子问题的特征（称之为状态）；

    2. 寻找每一个状态的可能决策，或者说是各状态间的相互转移方式（用数学的语言描述就是状态转移方程）。

    3. 按顺序求解每一个阶段的问题。

##最长公共子序列（Longest Common Subsequence，LCS）
给定一个长度为 $n$ 的序列 $A$ 和一个长度为 $m$ 的序列 $B$（$n$，$m$ $\leqslant$ 5000），求出一个最长的序列，使得该序列既是 $A$ 的子序列，也是 $B$ 的子序列。

解答：

设 $f(i, j)$ 表示只考虑 $A$ 的前 $i$ 个元素，$B$ 的前 $j$ 个元素时的最长公共子序列的长度，求这时的最长公共子序列的长度就是子问题。$f(i, j)$ 就是我们所说的 状态，则 $f(n, m)$ 是最终要达到的状态，即为所求结果。

对于每个 $f(i, j)$，存在三种决策：如果 $A_{i} = B_{j}$，则可以将它接到公共子序列的末尾；另外两种决策分别是跳过 $A_{i}$ 或者 $B_{j}$。状态转移方程如下：

$f(i, j) = \left \{ \begin{array}{l}
\hspace{0.5em} f(i - 1, j - 1),A_{i} = B_{j}\\
\hspace{0.5em} \max (f(i - 1, j),f(i, j - 1)),A_{i} \neq B_{j}
\end{array} \right .$

```cpp
int n, m, a[MAXN], b[MAXM], f[MAXN][MAXM];

int dp() {
  for (int i = 1; i <= n; i++)
    for (int j = 1; j <= m; j++)
      if (a[i] == b[j])
        f[i][j] = f[i - 1][j - 1] + 1;
      else
        f[i][j] = max(f[i - 1][j], f[i][j - 1]);
  return f[n][m];
}
```

时间复杂度：$O(nm)$

空间复杂度：$O(nm)$

###优化（空间复杂度）
$f[i][j]$ 依赖的三个值：$f[i-1][j-1]$（上一行前一列）、$f[i-1][j]$（上一行当前列）、$f[i][j-1]$（当前行前一列）。

第 $i-2$ 行及更早的信息，在计算第 $i$ 行时完全用不到，因此可以被覆盖掉,我们可以尝试用滚动数组来优化。

设 $dp[j]$ 代码表当前循环到序列 $A$ 的第 $i$ 个字符时的答案，即经典算法中的 $f[i][j]$。在更新 $dp[j]$ 时，$dp[j]$ 中保留的数据应该是 $dp[i - 1][j]$，$dp[j - 1]$ 中保留的数据是 $dp[i][j - 1]$。

但有个关键问题：当计算 $f[i][j]$ 时，$f[i-1][j-1]$（上一行前一列）会被当前行的 $f[i][j-1]$ 覆盖（因为 $dp[j-1]$ 会先被更新）。因此需要一个临时变量 $prev$ 来保存被覆盖前的 $f[i-1][j-1]$，故问题得到了解决。

```cpp
int n, m, a[MAXN], b[MAXM];

int dp() {
    // 选择较小的维度作为一维数组长度，进一步节省空间
    if (n < m) {
        // 交换 a 和 b，确保 m 是较小的维度
        swap(a, b);
        swap(n, m);
    }
    
    vector<int> dp(m + 1, 0);  // 一维数组，长度为 m+1（较小的维度）
    int prev = 0;  // 保存 f[i-1][j-1] 的值
    
    for (int i = 1; i <= n; i++) {
        prev = 0;  // 每轮 i 开始时，prev 对应 j=0 时的 f[i-1][0] = 0
        for (int j = 1; j <= m; j++) {
            int temp = dp[j];  // 暂存当前 dp[j]（即 f[i-1][j]），下一轮作为 prev
            if (a[i] == b[j]) {
                dp[j] = prev + 1;  // 等价于 f[i][j] = f[i-1][j-1] + 1
            } else {
                dp[j] = max(dp[j], dp[j-1]);  // 等价于 max(f[i-1][j], f[i][j-1])
            }
            prev = temp;  // 更新 prev 为下一个 j 的 f[i-1][j-1]
        }
    }
    
    return dp[m];
}
```

空间复杂度：$O(\min (n, m))$

##最长不下降子序列（Longest Increasing Subsequence，LIS）
给定一个长度为 $n$ 的序列 $a (n \leqslant 5000)$，求出一个最长的 $a$ 的子序列，满足该子序列的后一个元素不小于前一个元素。

设 $f(i)$ 表示以 $a_{i}$ 为结尾的最长不下降子序列的长度，则所求为 $\displaystyle \max_{1 \leqslant i \leqslant n} f(i)$。

计算 $f(i)$ 时，尝试将 $a_{i}$ 接到其他的最长不下降子序列后面，以更新答案。于是写出这样的状态转移方程：

$f(i) = \displaystyle \max_{1 \leqslant j \leqslant i, a_{i} \leqslant a_{i}} (f(j) + 1) $。

```cpp
int n, a[MAXN], d[MAXN];

int dp() {
  d[1] = 1;
  int ans = 1;
  for (int i = 2; i <= n; i++) {
    d[i] = 1;
    for (int j = 1; j < i; j++)
      if (a[j] <= a[i]) {
        d[i] = max(d[i], d[j] + 1);
        ans = max(ans, d[i]);
      }
  }
  return ans;
}
```

时间复杂度：$O(n^{2})$

空间复杂度：$O(n)$

###优化（时间复杂度）
考虑之前定义的状态 $(i, l)$，表示序列以第 $i$ 个元素结尾的不下降子序列最长为 $l$。不同于以往按固定 $i$ 处理状态的方法，这里直接判断 $(i, l)$ 是否合法：

- 初始状态 $(1,1)$ 必然合法。

- 对于任意 $(i, l)$，如果存在 $j < i$ 且 $(j, l - 1)$ 合法，同时 $a_{j} \leqslant a_{i}$，则 $(i, l)$ 合法。

最终，只需要找到合法状态中 $l$ 最大的 $(i, l)$，即可得到最长不下降子序列的长度。

设原序列为 $a_{1}, \cdots, a_{n}$，定义数组 $d$，其中第 $x$ 位表示长度为 $x$ 的不下降子序列末尾元素的最小值。初始时序列为空。令 $i$ 从 $1$ 到 $n$ 遍历，依次求出前 $i$ 个元素的最长不下降子序列的长度。对于当前元素 $a_{i}$：

- 如果 $a_{i}$ 大于等于序列$d$中最后一个元素，直接将元素 $a_{i}$ 插入到序列 $d$ 的末尾。
    
    - 解释：若 $a_{i}$ 大于等于当前最长子序列的末尾元素，说明存在一个不下降子序列可以接上 $a_{i}$。不插入将破坏最优性。

- 如果 $a_{i}$ 严格小于 $d$ 中最后一个元素，找到第一个大于它的元素，并用 $a_{i}$ 替换它。

    - 解释：若直接插在末尾，会破坏 $d$ 的单调性；替换操作可以保证每个长度的末尾元素尽可能小，从而为后续元素保留更多可能性。

    - 优化：因为 $d$ 单调不减，可用二分查找直接找到元素的插入位置，将整体复杂度降低到 $O(n \log n)$ 而非暴力查找的 $O(n^{2})$。

如果还要输出具体的最长不下降子序列，可以额外维护数组 $d'_{x}$，表示长度为 $x$ 的不下降子序列中末尾最小元素的位置（有多个可任选一个）。具体维护时，只需要在插入元素 $a_{i}$ 到 $d_{x}$ 时，同时更新 $d'_{x}$ 为 $i$ 即可。同时，需要记录 $i$ 的最优前驱 $p_{i}$ 为 $d'_{x - 1}$。最终，从任意最大长度状态出发，沿前驱 $p_{i}$ 回溯，即可得到完整子序列。

```cpp
int n, a[MAXN], d[MAXN], di[MAXN], pre[MAXN], res[MAXN];

int dp() {
  int ans = 0;
  for (int i = 1; i <= n; ++i) {
    int tmp = std::upper_bound(d, d + ans, a[i]) - d;
    pre[i] = tmp ? di[tmp - 1] : -1;
    d[tmp] = a[i];
    di[tmp] = i;
    if (tmp == ans) ++ans;
  }
  // 构建子序列
  for (int k = ans, i = di[ans - 1]; k; --k) {
    res[k] = a[i];
    i = pre[i];
  }
  return ans;
}
```
时间复杂度:$O(n \log n)$

输出答案的复杂度：$O(ans)$

空间复杂度：$O(n)$

注意：
- 对于最长上升子序列问题，类似地，可以令 $d_{i}$ 表示所有长度为 $i$ 的最长上升子序列的末尾元素的最小值。

- 需要注意的是，在第二步中，若 $a_{i} \leqslant d_{len}$，由于最长上升子序列中相邻元素不能相等，需要在 $d$ 序列中找到第一个不小于 $a_{i}$ 的元素，用 $a_{i}$ 替换之。

- 在实现上（以 C++ 为例），需要将 upper_bound 函数改为 lower_bound。



